In [1]:
using CSV, FileIO, DataFrames, Dates
using Plots, StatsPlots

In [2]:
region="italy"
catalog="ingv"
mkpath("./results/$region/")

"./results/italy"

In [3]:
df = CSV.read("./catalogs/$catalog.csv", DataFrame);

In [4]:
# Using describe function to get statistics of a dataset
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Any,Any,Int64,DataType
1,Datetime,,1985-01-02T18:39:30.740,2015-05-11T22:01:58.240,2023-03-08T11:05:17.900,0,DateTime
2,Latitude,42.2295,-69.757,42.8268,84.3258,0,Float64
3,Longitude,13.3849,-179.99,13.1407,179.939,0,Float64
4,Depth,12.8846,0.0,10.0,675.4,0,Float64
5,Magnitude_Type,,M,,mb,0,String7
6,Magnitude,1.58785,0.1,1.4,9.0,0,Float64


In [5]:
italy = df[(df.Longitude .>= 6.6166 ) .& (df.Longitude .<= 18.5166) .&
				  (df.Latitude .>=35.5) .& (df.Latitude .<= 47.0833) ,:]

Row,Datetime,Latitude,Longitude,Depth,Magnitude_Type,Magnitude
,DateTime,Float64,Float64,Float64,String7,Float64
1,1985-01-02T22:57:43.090,41.715,13.941,5.8,Md,2.6
2,1985-01-03T00:46:16.990,41.677,13.931,6.1,Md,2.5
3,1985-01-04T16:57:40.240,46.158,7.174,10.0,M,3.8
4,1985-01-04T23:59:54.820,45.338,6.641,5.0,M,3.7
5,1985-01-05T03:47:03.640,45.409,6.627,10.0,M,3.2
6,1985-01-05T08:32:41.900,43.056,13.139,2.8,Md,2.6
7,1985-01-05T10:13:26.800,45.15,7.86,20.8,Md,2.2
8,1985-01-06T03:52:23.380,43.357,12.537,10.0,Md,2.4
9,1985-01-06T04:19:36.650,43.37,12.545,10.0,Md,2.3


In [8]:
# Using describe function to get statistics of a dataset
describe(italy)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Any,Any,Int64,DataType
1,Datetime,,1985-01-02T22:57:43.090,2015-06-26T06:06:24.650,2023-03-08T11:05:17.900,0,DateTime
2,Latitude,42.4672,35.501,42.8325,47.083,0,Float64
3,Longitude,13.1058,6.6168,13.1372,18.513,0,Float64
4,Depth,12.5025,0.0,10.0,644.4,0,Float64
5,Magnitude_Type,,M,,mb,0,String7
6,Magnitude,1.53748,0.1,1.4,6.5,0,Float64


In [7]:
# Make directory if it does not exist
mkpath("./data/")

# Write to CSV
CSV.write("./data/$region.csv", italy)

"./data/italy.csv"